# SYSTEM

In [2]:
%load_ext memory_profiler
%load_ext snakeviz
%load_ext cython
%load_ext autoreload
%autoreload 2


from IPython.core import debugger
ist = debugger.set_trace

In [3]:
from py.typyMagics import *
ipy = get_ipython()
ipy.register_magics(typyMagics)

In [4]:
import sys
sys.path.insert(0,'../')

## DEFINE

In [5]:
%%run_and_write ../typyPRISM/core/System.py
#!python 
from __future__ import division,print_function
import numpy as np
from typyPRISM.core.PRISM import PRISM
from typyPRISM.core.MatrixArray import MatrixArray
from typyPRISM.core.PairTable import PairTable
from typyPRISM.core.ValueTable import ValueTable
from typyPRISM.core.Space import Space

class System:
    '''Primary class used to spawn PRISM calculations
    
    .. warning::
    
        The *intra*-molecular correlation functions (omega attribute)
        should be specified such that they are in Fourier space and such
        that their k->0 values approach the total number of sites in a 
        given molecule for the self (i==j) pairs.
    
    Attributes
    ----------
    types: list
        list of site types
        
    rank: int
        number of site types
    
    density: typyPRISM.core.ValueTable
        Table of site *number* density values
        
    potential: typyPRISM.core.PairTable
        Table of pair potentials between all site pairs in real space
        
    closure: typyPRISM.core.PairTable
        Table of closures between all site pairs
        
    omega: typyPRISM.core.PairTable
        Table of omega correlation functions in k-space
    
    domain: typyPRISM.core.Domain
        Domain object which specifies the Real and Fourier space 
        solution grid.
        
    kT: float
        Value of the thermal energy scale. Used to vary temperature and
        scale the potential energy functions.
    
    
    '''
    def __init__(self,types,kT=1.0):
        self.types = types
        self.rank  = len(types)
        self.kT = kT
        
        self.domain    = None
        self.density   = ValueTable(types,'density')
        self.potential = PairTable(types,'potential')
        self.closure   = PairTable(types,'closure')
        self.omega = PairTable(types,'omega')
    
    def check(self):
        '''Make sure all values in the system are specified'''
        for table in [self.density,self.potential,self.closure,self.omega]:
            table.check()
        
        if self.domain is None:
            raise ValueError(('System has no domain! '
                              'User must instatiate and assign a domain to the system!'))
    def createDensityMatrices(self):
        '''See method name
        
        .. math::
        
            \rho^{pair}_{i,j} = \rho_i * \rho_j
            
        
            \rho^{site}_{i,j} = \rho_i + \rho_j, if i != j
            
            \rho^{site}_{i,j} = \rho_i         , if i = j
        '''
        siteDensityMatrix = np.zeros((self.rank,self.rank))
        pairDensityMatrix = np.zeros((self.rank,self.rank))
        for i,t1,rho1 in self.density:
            for j,t2,rho2 in self.density:
                if i>j:
                    continue
                    
                if i==j:
                    siteDensityMatrix[i,j] = rho1
                    pairDensityMatrix[i,j] = rho1 * rho2
                else:
                    siteDensityMatrix[i,j] = rho1 + rho2
                    siteDensityMatrix[j,i] = rho1 + rho2
                    
                    pairDensityMatrix[i,j] = rho1 * rho2
                    pairDensityMatrix[j,i] = rho1 * rho2
                    
        return siteDensityMatrix,pairDensityMatrix
            
    def createPRISM(self):
        '''Construct a fully specified PRISM object that can be solved'''
        
        self.check() #sanity check

        #create density
        siteDensityMatrix,pairDensityMatrix = self.createDensityMatrices()
        
        
        # The omega objects must be converted to arrays of the actual correlation
        # function values. 
        omega = self.omega.apply(lambda x: x.calculate(self.domain.k),inplace=False)
        
        # Next, the omega table is converted to a MatrixArray so that we can easily
        # do operations during the PRISM solution
        omega = omega.exportToMatrixArray(space=Space.Fourier)
        
        # Finally, the correlation functions are scaled by the density matrix
        omega *= siteDensityMatrix 
        
        # Need to set the omega potential for each closure object
        for (i,j),(t1,t2),U in self.potential.iterpairs():
            self.closure[t1,t2].potential = U.calculate(self.domain.r) / self.kT
        
        kwargs = {}
        kwargs['kT'] = self.kT
        kwargs['rank'] = self.rank
        kwargs['domain'] = self.domain
        kwargs['closure'] = self.closure
        kwargs['omega'] = omega
        kwargs['pairDensityMatrix'] = pairDensityMatrix
        kwargs['siteDensityMatrix'] = siteDensityMatrix
        return PRISM(**kwargs)
        

Overwriting ../typyPRISM/core/System.py


In [6]:
%%run_and_write ../typyPRISM/test/System_TestCase.py
#!python
from __future__ import division,print_function
import typyPRISM
import unittest
import numpy as np

class System_TestCase(unittest.TestCase):
    def test_create(self):
        '''Can we create a system?'''
        types = ['A','B','C']
        sys = typyPRISM.System(types)
    def test_check(self):
        '''Can we validate a system object?'''
        types = ['A','B','C']
        sys = typyPRISM.System(types)
        self.assertRaises(ValueError,sys.check)
        
        # These values make no sense but their sufficient
        # for this 'check'
        sys.domain = 'dummy'
        sys.density.setUnset(1.0)
        sys.potential.setUnset(1.0)
        sys.closure.setUnset(1.0)
        sys.omega.setUnset(1.0)
        
        #test should fail if this raises
        try:
            sys.check()
        except ValueError:
            self.fail('Check failed when it should have passed...')
    def test_createPRISM(self):
        '''Can we construct a simple but fully specified PRISM problem?'''
        
        sys = typyPRISM.System(['A','B'])
        
        sys.domain = typyPRISM.Domain(dr=0.1,length=1024)
        
        sys.density['A'] = 0.45
        sys.density['B'] = 0.35
        
        sys.closure.setUnset(typyPRISM.closure.PercusYevick())
        
        sys.potential.setUnset(typyPRISM.potential.HardSphere(sigma=1.0))
        
        sys.omega['A','A'] = typyPRISM.omega.SingleSite()
        sys.omega['A','B'] = typyPRISM.omega.NoIntra()
        sys.omega['B','B'] = typyPRISM.omega.Gaussian(sigma=1.0,length=100)
        
        PRISM = sys.createPRISM()
        
        self.assertIsInstance(PRISM,typyPRISM.core.PRISM.PRISM)
        
    def test_createDensityMatrices(self):
        '''Can we construct the two necesssary density matrices?'''
        sys = typyPRISM.System(['A','B'])
        
        sys.density['A'] = rhoA = 0.45
        sys.density['B'] = rhoB = 0.35
        
        siteDensityMatrix1 = np.zeros((2,2))
        siteDensityMatrix1[0,0] = rhoA
        siteDensityMatrix1[0,1] = rhoA + rhoB
        siteDensityMatrix1[1,0] = rhoA + rhoB
        siteDensityMatrix1[1,1] = rhoB
        
        pairDensityMatrix1 = np.zeros((2,2))
        pairDensityMatrix1[0,0] = rhoA * rhoA
        pairDensityMatrix1[0,1] = rhoA * rhoB
        pairDensityMatrix1[1,0] = rhoB * rhoA
        pairDensityMatrix1[1,1] = rhoB * rhoB
        
        siteDensityMatrix2,pairDensityMatrix2 = sys.createDensityMatrices()
        
        np.testing.assert_array_almost_equal(siteDensityMatrix1,siteDensityMatrix2)
        np.testing.assert_array_almost_equal(pairDensityMatrix1,pairDensityMatrix2)
        
        
        
         
        


Overwriting ../typyPRISM/test/System_TestCase.py


In [7]:
import unittest
suite = []
suite.append(unittest.TestLoader().loadTestsFromTestCase(System_TestCase))
suite = unittest.TestSuite(suite)
unittest.TextTestRunner(verbosity=2).run(suite)

test_check (__main__.System_TestCase)
Can we validate a system object? ... ok
test_create (__main__.System_TestCase)
Can we create a system? ... ok
test_createDensityMatrices (__main__.System_TestCase)
Can we construct the two necesssary density matrices? ... ok
test_createPRISM (__main__.System_TestCase)
Can we construct a simple but fully specified PRISM problem? ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.010s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>